In [27]:
import pickle

In [28]:
import os

In [29]:
os.getcwd()

'/lustre/work/statgrads/vpiyush2/MEnKF-ANN/Data_Generation/LSTM_Low_Dropout'

In [30]:
with open('../Data//train_valid_test_splits_50.pkl', 'rb') as f:
    catch = pickle.load(f)

In [31]:
with open('Low_Data_train_logits.pkl', 'rb') as f:
    catch_train_logits = pickle.load( f)

In [32]:
with open('Low_Data_valid_logits.pkl', 'rb') as f:
    catch_valid_logits = pickle.load( f)

In [33]:
# catch_valid_logits[0]

In [34]:
# second_lstm[0][0]

In [35]:
with open('Low_Data_test_logits.pkl', 'rb') as f:
    catch_test_logits = pickle.load( f)

In [36]:
var_targets = 0.0001

In [37]:
reps = 50

In [38]:
from scipy.stats import norm

In [39]:
import tensorflow as tf

In [40]:
catch_train_logits_second = []
catch_train_probs_second = []
catch_train_labels_second = []
for i in range(0, reps):
    train_true_logits = catch_train_logits[i]
    train_logits_noise = norm(0,var_targets).rvs(train_true_logits.shape[0]).reshape(-1,1)
    train_logits_second = train_true_logits + train_logits_noise
    train_fudged_probs = tf.nn.sigmoid(train_logits_second).numpy()
    train_fudged_labels = (train_fudged_probs >= 0.5).astype(float)
    # train_fudged_labels = np.random.binomial(1, p=train_fudged_probs)
    catch_train_logits_second.append(train_logits_second)
    catch_train_probs_second.append(train_fudged_probs)
    catch_train_labels_second.append(train_fudged_labels)

In [41]:
catch_valid_logits_second = []
catch_valid_probs_second = []
catch_valid_labels_second = []
for i in range(0, reps):
    valid_true_logits = catch_valid_logits[i]
    valid_logits_noise = norm(0,var_targets).rvs(valid_true_logits.shape[0]).reshape(-1,1)
    valid_logits_second = valid_true_logits + valid_logits_noise
    valid_fudged_probs = tf.nn.sigmoid(valid_logits_second).numpy()
    valid_fudged_labels = (valid_fudged_probs >= 0.5).astype(float)
    catch_valid_logits_second.append(valid_logits_second)
    catch_valid_probs_second.append(valid_fudged_probs)
    catch_valid_labels_second.append(valid_fudged_labels)

In [42]:
catch_test_logits_second = []
catch_test_probs_second = []
catch_test_labels_second = []
for i in range(0, reps):
    test_true_logits = catch_test_logits[i]
    test_logits_noise = norm(0,var_targets).rvs(test_true_logits.shape[0]).reshape(-1,1)
    test_logits_second = test_true_logits + test_logits_noise
    test_fudged_probs = tf.nn.sigmoid(test_logits_second).numpy()
    test_fudged_labels = (test_fudged_probs >= 0.5).astype(float)
    # test_fudged_labels = np.random.binomial(1, p=test_fudged_probs)
    catch_test_logits_second.append(test_logits_second)
    catch_test_probs_second.append(test_fudged_probs)
    catch_test_labels_second.append(test_fudged_labels)

In [43]:
import numpy as np

In [44]:
for i in range(0, reps):
    print(np.mean(catch_train_labels_second[i] == catch[i][3].reshape(-1,1)))

0.9411764705882353
0.9019607843137255
0.9411764705882353
0.9019607843137255
0.9411764705882353
0.8627450980392157
0.9215686274509803
0.9411764705882353
0.9607843137254902
0.9411764705882353
0.9803921568627451
0.9215686274509803
0.9607843137254902
0.9411764705882353
0.9411764705882353
0.9803921568627451
0.9803921568627451
0.9607843137254902
0.9215686274509803
0.9803921568627451
0.9607843137254902
0.9215686274509803
1.0
0.9803921568627451
0.9411764705882353
0.9411764705882353
0.9019607843137255
0.9215686274509803
0.9803921568627451
0.9607843137254902
0.9607843137254902
0.9607843137254902
1.0
0.9803921568627451
0.9019607843137255
0.9215686274509803
0.9411764705882353
0.9803921568627451
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9803921568627451
0.9607843137254902
0.9803921568627451
0.9607843137254902
0.9411764705882353
0.9019607843137255
0.9607843137254902
0.9607843137254902
0.9411764705882353


In [45]:
rate = 0.5

In [46]:
import gensim

In [47]:
model_cbow = gensim.models.word2vec.Word2Vec.load(r"..//word2vec_sg")

In [48]:
def attention_lstm_model(rate, training): 
    padding_vector = np.zeros((1, model_cbow.wv.vectors.shape[1]))
    unknown_vector = np.zeros((1, model_cbow.wv.vectors.shape[1]))
    weight_vectors = np.vstack((padding_vector, unknown_vector))
    weight_vectors = np.vstack((weight_vectors, model_cbow.wv.vectors))
    embedding_layer = tf.keras.layers.Embedding(len(weight_vectors),
                            weight_vectors.shape[1],
                            weights=[weight_vectors],
                            mask_zero = False,
                            trainable=False)

    
    vectorize_layer = tf.keras.layers.TextVectorization(
                     output_mode='int',
                     vocabulary=model_cbow.wv.index_to_key, 
                     standardize = None)
    
    input_layer = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

    vectorize = vectorize_layer(input_layer)

    vectorize.shape

    emb_output = embedding_layer(vectorize)

    emb_output.shape
    
    # spatial_drop = tf.keras.layers.SpatialDropout1D(0.5)
    
    # spatial_out = spatial_drop(emb_output, training = training)

    lstm_layer = tf.keras.layers.LSTM(25, return_sequences = True, dropout = rate)

    lstm_output = lstm_layer(emb_output, training = training)

#     x_a = tf.keras.layers.Dense(lstm_output.get_shape()[-1]//2, kernel_initializer = 'glorot_uniform', activation="tanh", name="tanh_mlp")(lstm_output) 
    
    # x_a = tf.keras.layers.SpatialDropout1D(0.75)(lstm_output, training = training)
    
    x_a = tf.keras.layers.Dense(1, kernel_initializer = 'glorot_uniform', activation='linear', name="word-level_context", kernel_regularizer=tf.keras.regularizers.L2())(lstm_output)

    x_a = tf.keras.layers.Flatten()(x_a)

    att_out = tf.keras.layers.Activation('softmax')(x_a) 

    x_a2 = tf.keras.layers.RepeatVector(lstm_output.get_shape()[-1])(att_out)

    x_a2 = tf.keras.layers.Permute([2,1])(x_a2)

    out = tf.keras.layers.Multiply()([lstm_output,x_a2])
    
    out = tf.keras.layers.Lambda(lambda x : tf.math.reduce_sum(x, axis = 1), name='expectation_over_words')(out)
    
    # dropout_layer = tf.keras.layers.Dropout(0.65)(out, training = training)

    pred_head = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L2())

    pred_output = pred_head(out)

    model = tf.keras.models.Model(input_layer, pred_output)
    
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                 optimizer = tf.keras.optimizers.Adam(learning_rate = lr), 
                 metrics=tf.keras.metrics.BinaryAccuracy())
    
    return model

In [49]:
lr = 1e-3

In [50]:
# chk = attention_lstm_model(0.5, False)

In [51]:
# @tf.function
def second_LSTM_training( idx):
#     @tf.function
    second_lstm = attention_lstm_model(rate, False)
    # second_lstm.set_weights(first_lstm[idx][-1])
    # second_lstm = simple_lstm(False, model_cbow)
    
#     train_idx = catch_train_idxes[idx]
    # valid_idx = catch_valid_idxes[idx]
#     test_idx = catch_test_idxes[idx]
    second_lstm.fit(catch[idx][0], catch_train_labels_second[idx], epochs = 2000, verbose = 0, 
                  callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 30,
                                                              restore_best_weights=True), 
                   validation_data = (catch[idx][1], catch_valid_labels_second[idx]))
    
    second_pred_train_logits = second_lstm.predict(catch[idx][0], verbose = 0)
    second_pred_valid_logits = second_lstm.predict(catch[idx][1], verbose = 0)
    second_pred_test_logits = second_lstm.predict(catch[idx][2], verbose = 0)
    
    lstm_extract = tf.keras.models.Model(second_lstm.input, second_lstm.layers[-2].output)
    
    train_lstm_extract = lstm_extract(catch[idx][0])
    valid_lstm_extract = lstm_extract(catch[idx][1])
    test_lstm_extract = lstm_extract(catch[idx][2])
    
    return second_pred_train_logits, second_pred_valid_logits, second_pred_test_logits, train_lstm_extract, valid_lstm_extract, test_lstm_extract

In [52]:
from joblib import Parallel, delayed

In [53]:
second_lstm = Parallel(n_jobs=15, verbose = 10, backend = "loky")(delayed(second_LSTM_training)( i) for i in range(reps))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
2023-04-10 01:53:56.021888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 01:53:56.021898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 01:53:56.021911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruct

In [54]:
with open('low_dropout_second_lstm.pkl', 'wb') as f:
    pickle.dump(second_lstm, f)